This script runs one param_set. Outputs results{}.csv and trialxtrial{}.csv

In [1]:
import sys
print("Python version")
print (sys.version)

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
from sklearn import metrics
import pandas as pd

from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from scipy.special import logsumexp
from scipy.stats import norm
from glob import glob

Python version
3.7.9 (default, Aug 31 2020, 07:22:35) 
[Clang 10.0.0 ]


In [2]:
from schema_prediction_task_9_8_20 import generate_exp, batch_exp
from vanilla_lstm import VanillaLSTM
from sem.event_models import NonLinearEvent

TensorFlow Version: 2.3.1


# Script parameters

### gridsearch params 

In [3]:
# parameter search over lr, n_epochs, alpha, lambda
model_type        = 'SEM'
lr                = 0.05
n_epochs          = 2    # what does this control?
log_alpha         = 0.0  # sCRP alpha is set in log scale
log_lambda        = 0.0  # sCRP lambda is set in log scale

In [4]:
# toggle between SEM (False) and LSTM (True)
if model_type == 'SEM':
  no_split=False
elif model_type == 'LSTM':
  no_split=True

### embedding params

In [5]:
# actor_weight & instructions_weight control instruction condition
story_kwargs = dict(seed=None, err=0.2, actor_weight=1.0, instructions_weight=0.0)

## to inspect embeddings, use:
# x, y, e, embedding_library = generate_exp('blocked', **story_kwargs)

### SEM configuration

In [6]:
optimizer_kwargs = dict(
    lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-5, amsgrad=False
) 

f_opts=dict(
  batch_size=25, 
  batch_update=False, 
  dropout=0.0,
  l2_regularization=0.0, 
  n_epochs=n_epochs,
    optimizer_kwargs=optimizer_kwargs
)

f_class = VanillaLSTM # event model class

# final param dict
sem_kwargs = dict(
  lmda=np.exp(log_lambda), 
  alfa=np.exp(log_alpha), 
  f_opts=f_opts, 
  f_class=f_class
)


# Run model

main fun call

In [7]:
""" condition params: improve """

block_only=True 
run_instructed=False 

In [15]:
""" 
batch_exp main fun call
"""

n_batch = 1 # remove or debug param?
print('n_batch',n_batch)
results, trialXtrial, _ = batch_exp(
              sem_kwargs, story_kwargs, no_split=no_split, 
              block_only=block_only, run_instructed=run_instructed, 
              sem_progress_bar=True, progress_bar=False,
              n_batch=n_batch, 
)


n_batch 1


Run SEM: 100%|██████████| 200/200 [01:26<00:00,  2.32it/s]


In [ ]:
# convert from JSON file format (dict) to pandas df
results = pd.DataFrame(results)
trialXtrial = pd.DataFrame(trialXtrial)

# save


In [9]:
save_fpath = None
results,trialXtrial

(     Trials  adjRand  nClusters        pe  pe (probes)  verb decoder Accuracy  \
 0       All      0.0          1  0.566003     0.581288               0.107500   
 1  Training      0.0          1  0.564398     0.580805               0.115625   
 2      Test      0.0          1  0.572425     0.583218               0.075000   
 
    verb decoder Accuracy Prob  verb 2 AFC decoder Prob  batch Condition  \
 0                    0.117275                 0.496754      0   Blocked   
 1                    0.124738                 0.500790      0   Blocked   
 2                    0.087423                 0.480609      0   Blocked   
 
    cluster re-use  
 0             NaN  
 1             NaN  
 2             1.0  ,
        t  e_hat  Accuracy        pe  batch Condition
 0      0      0  0.491652  0.619152      0   Blocked
 1      1      0  0.163109  0.532164      0   Blocked
 2      2      0  0.294964  0.717002      0   Blocked
 3      3      0  0.454531  0.613671      0   Blocked
 4      4

In [16]:

model_tag = "%s-lr-%.3f-nepchs-%i-alpha-%.3f-lambda-%.3f"%(
  model_type,lr,n_epochs,log_alpha,log_lambda
)
print(model_tag)


LSTM-lr-0.050-nepchs-2-alpha-0.000-lambda-0.000


In [13]:
results_fpath = "results_" + model_tag + '.csv'
trial_fpath = "trial_X_trial_" + model_tag + '.csv'

save_dir = ''